In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
torch.cuda.current_device()

0

In [3]:
torch.cuda.get_device_name(0)

'GeForce RTX 3060 Laptop GPU'

## Sending Models to GPU

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    
        self.fc2 = nn.Linear(h1, h2)            
        self.out = nn.Linear(h2, out_features)  
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [6]:
torch.manual_seed(32)
model = Model()

In [7]:
next(model.parameters()).is_cuda

False

In [8]:
gpumodel = model.cuda()

In [9]:
next(gpumodel.parameters()).is_cuda

True

In [11]:
df = pd.read_csv('iris.csv')
X = df.drop('target',axis=1).values
y = df['target'].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=33)

## Converting Tensors to .cuda() tensors

In [12]:
X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()
y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()

In [13]:
trainloader = DataLoader(X_train, batch_size=60, shuffle=True)
testloader = DataLoader(X_test, batch_size=60, shuffle=False)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [15]:
import time
epochs = 1000
losses = []
start = time.time()
for i in range(epochs):
    i+=1
    y_pred = gpumodel.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)
    
    # a neat trick to save screen space:
    if i%10 == 1:
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print(f'TOTAL TRAINING TIME: {time.time()-start}')

epoch:  1  loss: 1.15069771
epoch: 11  loss: 0.93772912
epoch: 21  loss: 0.77980196
epoch: 31  loss: 0.61059004
epoch: 41  loss: 0.40147230
epoch: 51  loss: 0.25468495
epoch: 61  loss: 0.15076803
epoch: 71  loss: 0.10109342
epoch: 81  loss: 0.08147227
epoch: 91  loss: 0.07254115
epoch: 101  loss: 0.06736170
epoch: 111  loss: 0.06382950
epoch: 121  loss: 0.06114199
epoch: 131  loss: 0.05885541
epoch: 141  loss: 0.05726259
epoch: 151  loss: 0.05572830
epoch: 161  loss: 0.05455634
epoch: 171  loss: 0.05339395
epoch: 181  loss: 0.05245451
epoch: 191  loss: 0.05167523
epoch: 201  loss: 0.05074298
epoch: 211  loss: 0.05007675
epoch: 221  loss: 0.04940847
epoch: 231  loss: 0.04880815
epoch: 241  loss: 0.04829729
epoch: 251  loss: 0.04784623
epoch: 261  loss: 0.04739534
epoch: 271  loss: 0.04690465
epoch: 281  loss: 0.04647555
epoch: 291  loss: 0.04620136
epoch: 301  loss: 0.04571205
epoch: 311  loss: 0.04541483
epoch: 321  loss: 0.04507039
epoch: 331  loss: 0.04477985
epoch: 341  loss: 0.0445